In [8]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0152,  0.0988,  0.0838,  0.0666, -0.3508, -0.0917, -0.0414,  0.1041,
         -0.1836,  0.0698],
        [ 0.0444,  0.1062,  0.0460,  0.0286, -0.3669, -0.0254,  0.0590,  0.0841,
         -0.1432,  0.0828]], grad_fn=<AddmmBackward>)

### 5.1.1 自定义块
要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。 在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能：
+ 将输入数据作为其前向传播的参数
+ 通过前向传播函数来生成输出。
+ 计算其输出关于输入的梯度
+ 存储和访问前向传播计算所需要的参数
+ 根据需要初始化模型参数

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, x):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [10]:
net = MLP()
net(X)

tensor([[ 0.0431,  0.4604,  0.0293,  0.2095, -0.0230, -0.0433,  0.0768, -0.0257,
         -0.0326,  0.2002],
        [-0.0325,  0.4500,  0.0673,  0.2576, -0.0843, -0.0553,  0.0459, -0.1746,
         -0.0044,  0.1473]], grad_fn=<AddmmBackward>)

### 5.1.2 顺序块
现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：
1. 一种将块诸葛追加到列表中的函数
2. 一种前向传播函数，用于将输入按追加快的顺序传递给块组成的“链条”

下面的MySequential类提供了与默认Sequential类相同的功能。

In [14]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [15]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0715,  0.0201, -0.1685, -0.0787,  0.0659, -0.0140, -0.2452, -0.0751,
         -0.0005,  0.1615],
        [ 0.0874,  0.2286, -0.1283, -0.0420,  0.0985, -0.0346, -0.2000, -0.0069,
         -0.0183,  0.0970]], grad_fn=<AddmmBackward>)

### 5.1.3 在前向传播函数中执行代码
Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP, self).__init__()
        # 不计算梯度的随机权重参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20,20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常量参数以及relu和mm函数
        x = F.relu(torch.mm(x, self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        x = self.linear(x)
        # 控制流
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在\(L_1\)范数大于\(1\)的条件下， 将输出向量除以\(2\)，直到它满足条件为止。 最后，模型返回了X中所有项的和。 注意，此操作可能不会常用于在任何实际任务中， 我们只是向你展示如何将任意代码集成到神经网络计算的流程中。

In [17]:
net = FixedHiddenMLP()
net(X)

tensor(0.0939, grad_fn=<SumBackward0>)

In [19]:
# 我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。
class NestMLP(nn.Module):
    def __init__(self):
        super(NestMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    def forward(self, x):
        return self.linear(self.net(x))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3544, grad_fn=<SumBackward0>)